### 1. Download the Data from Kaggle and Unzip


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fatmanuryaman","key":"e1244bfe070d16323b3d571469e74df4"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'maedemaftouni/large-covid19-ct-slice-dataset'

 99% 2.05G/2.06G [00:24<00:00, 145MB/s]
100% 2.06G/2.06G [00:24<00:00, 91.4MB/s]


### 2. Splitting

In [ ]:
import pandas as pd

meta_normal = pd.read_csv("/tmp/meta_data_normal.csv")
meta_covid = pd.read_csv("/tmp/meta_data_covid.csv", encoding='windows-1252')
meta_cap = pd.read_csv("/tmp/meta_data_cap.csv")

# Define the variables below using meta dataframes

normal_pt_nb = len(pd.unique(meta_normal["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in normal group
covid_pt_nb = len(pd.unique(meta_covid["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in covid group
cap_pt_nb = len(pd.unique(meta_cap["Patient ID"])) ##### INSERT YOUR CODE HERE ##### # Number of patients in CAP group

normal_img_nb = len(meta_normal) ##### INSERT YOUR CODE HERE ##### # Number of images in normal group
covid_img_nb = len(meta_covid) ##### INSERT YOUR CODE HERE ##### # Number of images in covid group
cap_img_nb = len(meta_cap) ##### INSERT YOUR CODE HERE ##### # Number of images in CAP group

print("Normal")
print("Number of patient: ", normal_pt_nb)
print("Number of image: ", normal_img_nb)

print("\nCovid")
print("Number of patient: ", covid_pt_nb)
print("Number of image: ", covid_img_nb)

print("\nCAP")
print("Number of patient: ", cap_pt_nb)
print("Number of image: ", cap_img_nb)


Normal
Number of patient:  604
Number of image:  6893

Covid
Number of patient:  464
Number of image:  7593

CAP
Number of patient:  54
Number of image:  2618


In [ ]:

import numpy as np

# Set seed to get the same result (I specifically chose this seed after a couple of tries so that we'll have approximately same split ratios on slice level as well)
np.random.seed(58)
val_split_size = .2
test_split_size = .5

normal_val_file_list, normal_test_file_list = [], []
covid_val_file_list, covid_test_file_list = [], []
##### START OF YOUR CODE #####
normal_pt_list = np.array(pd.unique(meta_normal["Patient ID"]))
np.random.shuffle(normal_pt_list)
normal_val_pts = normal_pt_list[:int(normal_pt_nb * val_split_size)]
normal_test_pts = normal_pt_list[int(normal_pt_nb * val_split_size): int(normal_pt_nb * (val_split_size+test_split_size))]

normal_val_file_list = []
for pt in normal_val_pts:
  for i in meta_normal["File name"][meta_normal["Patient ID"] == pt].values:
    normal_val_file_list.append(i)

normal_test_file_list = []
for pt in normal_test_pts:
  for i in meta_normal["File name"][meta_normal["Patient ID"] == pt].values:
    normal_test_file_list.append(i)

covid_pt_list = np.array(pd.unique(meta_covid["Patient ID"]))
np.random.shuffle(covid_pt_list)
covid_val_pts = covid_pt_list[:int(covid_pt_nb * val_split_size)]
covid_test_pts = covid_pt_list[int(covid_pt_nb * val_split_size): int(covid_pt_nb * (val_split_size+test_split_size))]

covid_val_file_list = []
for pt in covid_val_pts:
  for i in meta_covid["File name"][meta_covid["Patient ID"] == pt].values:
    covid_val_file_list.append(i)

covid_test_file_list = []
for pt in covid_test_pts:
  for i in meta_covid["File name"][meta_covid["Patient ID"] == pt].values:
    covid_test_file_list.append(i)
##### END OF YOUR CODE #####

print("Slice-based val size: ")
print("Normal: ", round(len(normal_val_file_list)/normal_img_nb, 2))
print("Covid: ", round(len(covid_val_file_list)/covid_img_nb, 2))

print("\nSlice-based test size: ")
print("Normal: ", round(len(normal_test_file_list)/normal_img_nb, 2))
print("Covid: ", round(len(covid_test_file_list)/covid_img_nb, 2))

Slice-based val size: 
Normal:  0.21
Covid:  0.18

Slice-based test size: 
Normal:  0.51
Covid:  0.5


In [ ]:
import os

##### START OF YOUR CODE #####
# Create train and test directories and move the files accordingly
[os.makedirs("/tmp/curated_data/data/"+x+y, exist_ok=True) for x in ["train/", "val/", "test/"] for y in ["normal/", "covid/"]]

# Normal group
for pt in os.listdir("/tmp/curated_data/curated_data/1NonCOVID/"):
  # validation
  if pt in normal_val_file_list:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/val/normal/")
  # test
  elif pt in normal_test_file_list:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/test/normal/")
  # train
  else:
    shutil.move("/tmp/curated_data/curated_data/1NonCOVID/"+pt, "/tmp/curated_data/data/train/normal/")

# Covid group
for pt in os.listdir("/tmp/curated_data/curated_data/2COVID/"):
  # validation
  if pt in covid_val_file_list:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/val/covid/")
  # test
  elif pt in covid_test_file_list:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/test/covid/")
  # train
  else:
    shutil.move("/tmp/curated_data/curated_data/2COVID/"+pt, "/tmp/curated_data/data/train/covid/")

##### END OF YOUR CODE #####

data_counts = {x+y: len(os.listdir("/tmp/curated_data/data/"+x+y)) for x in ["train/", "val/", "test/"] for y in ["normal/", "covid/"]}
for i in ["train", "val", "test"]:
  print("\nPercentage of {} set: {:.2f}" .format(i, (data_counts[i+"/normal/"]+data_counts[i+"/covid/"])/sum(data_counts.values())))
  print("Percentage of Covid + slices in {} set is: {:.2f}" .format(i, data_counts[i+"/covid/"]/(data_counts[i+"/normal/"]+data_counts[i+"/covid/"])))

### 3. Dataloader

In [ ]:
!pip install -q torchio

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
import torchio as tio
import torchvision.transforms as T


class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, image_size, data_folder, partition):
    ##### START OF YOUR CODE #####
    self.image_size = image_size
    self.partition = partition
    self.data_folder = data_folder
    self.paths = self.img_paths()

    ##### END OF YOUR CODE #####
  def __len__(self):
    ##### START OF YOUR CODE #####
    return len(self.paths)
    ##### END OF YOUR CODE #####

  def __getitem__(self, idx):
    ##### START OF YOUR CODE #####
      img = self.read_and_resize_img(self.paths[idx])
      label = int(self.paths[idx].rsplit("/", 2)[1] == "covid")
      label = np.array(label)[np.newaxis]

      if self.partition == "train":
        img = self.augmentation(img)
      return (img, label)
    ##### END OF YOUR CODE #####

  def img_paths(self):
    ##### START OF YOUR CODE #####
    normal_paths = [os.path.join(self.data_folder, self.partition, "normal", i) for i in os.listdir(os.path.join(self.data_folder, self.partition, "normal"))]
    covid_paths = [os.path.join(self.data_folder, self.partition, "covid", i) for i in os.listdir(os.path.join(self.data_folder, self.partition, "covid"))]
    paths = normal_paths + covid_paths
    np.random.shuffle(paths)
    return paths
    ##### END OF YOUR CODE #####

  def read_and_resize_img(self, path):
    ##### START OF YOUR CODE #####
    img = Image.open(path).convert('L')
    img = np.array(img)[np.newaxis]/255.

    if (self.image_size, self.image_size) != img.shape[1:]:
      resizing = T.Resize((self.image_size, self.image_size), antialias=False)
      resized_img = resizing(torch.Tensor(img))
      return resized_img
    return torch.Tensor(img)
    ##### END OF YOUR CODE #####

  def augmentation(self, data):
    ##### START OF YOUR CODE #####
    transform = tio.transforms.OneOf({
        tio.transforms.OneOf({
            tio.transforms.RandomNoise(): .25,
            tio.transforms.RandomBiasField(): .25,
            tio.transforms.RandomGhosting(): .25,
            tio.transforms.RandomSpike(): .25,
            tio.transforms.RandomAffine(degrees=10, scales=0., translation=0.): .25
        }): .8
    })
    aug_data = torch.squeeze(transform(torch.unsqueeze(data, -1)), -1)
    return aug_data
    ##### END OF YOUR CODE #####


### 4. ResNet-18

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


class ConvBlock(nn.Module):
    def __init__(self, ch_in, ch_out, s, act):
      super(ConvBlock,self).__init__()
      # Initialize layers
      ##### START OF YOUR CODE #####
      if act == "relu":
          self.act_layer = nn.ReLU()
      elif act == "leaky_relu":
          self.act_layer = nn.LeakyReLU()
      elif act == "gelu":
          self.act_layer = nn.GELU()

      self.conv1 = nn.Conv2d(ch_in, ch_in, kernel_size=3, stride=s, padding=1)
      self.bn1 = nn.BatchNorm2d(ch_in)
      self.conv2 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
      self.bn2 = nn.BatchNorm2d(ch_out)

      self.conv_sc = nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=s)
      self.bn_sc = nn.BatchNorm2d(ch_out)
      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####
      X_shortcut = X
      X = self.bn1(self.conv1(X))
      X = self.act_layer(X)
      X = self.bn2(self.conv2(X))

      X_shortcut = self.bn_sc(self.conv_sc(X_shortcut))
      X = self.act_layer(X + X_shortcut)
      ##### END OF YOUR CODE #####
      return X


class ResNet18(nn.Module):
    def __init__(self, act, drop_rate,image_size):
      super(ResNet18, self).__init__()
      # Initialize layers
      ##### START OF YOUR CODE #####
      if act == "relu":
          self.act_layer = nn.ReLU()
      elif act == "leaky_relu":
          self.act_layer = nn.LeakyReLU()
      elif act == "gelu":
          self.act_layer = nn.GELU()

      self.image_size = image_size

      self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3) # 64 x N/2 x N/2
      self.bn1 = nn.BatchNorm2d(64)

      self.mp = nn.MaxPool2d((3, 3), stride=2, padding=1) # 64 x N/4 x N/4
      self.conv2a = ConvBlock(64, 64, 1, act)
      self.conv2b = ConvBlock(64, 64, 1, act) # 64 x N/4 x N/4

      self.conv3a = ConvBlock(64, 128, 2, act)
      self.conv3b = ConvBlock(128, 128, 1, act) # 128 x N/8 x N/8

      self.conv4a = ConvBlock(128, 256, 2, act)
      self.conv4b = ConvBlock(256, 256, 1, act) # 256 x N/16 x N/16

      self.conv5a = ConvBlock(256, 512, 2, act)
      self.conv5b = ConvBlock(512, 512, 1, act) # 512 x N/32 x N/32

      kernel_size = self.image_size /(2**5)
      self.avgpool = nn.AvgPool2d((int(kernel_size), int(kernel_size)))
      self.flat = nn.Flatten()
      self.dropout = nn.Dropout(drop_rate)
      self.fc = nn.Linear(512, 1)
      ##### END OF YOUR CODE #####

    def forward(self, X):
      ##### START OF YOUR CODE #####
      batch_size = X.size(0)
      image_size = X.size(2)

      X = self.conv1(X)
      X = self.bn1(X)
      X = self.act_layer(X)

      X = self.mp(X)
      X = self.conv2a(X)
      X = self.conv2b(X)

      X = self.conv3a(X)
      X = self.conv3b(X)

      X = self.conv4a(X)
      X = self.conv4b(X)

      X = self.conv5a(X)
      X = self.conv5b(X)

      X = self.avgpool(X)
      X = self.flat(X)
      X = self.fc(self.dropout(X))
      X = torch.sigmoid(X)
      ##### END OF YOUR CODE #####
      return X

# Assignment 3
<p>In this assignment you will

* write helper functions
* train the model
* hyperparameter search using W&B

Read the comments carefully and insert your code where you see: <br><br><b>##### START OF YOUR CODE #####</b><br><br><b>##### END OF YOUR CODE #####</b><br><br>or for the inline codes you will see<br><br><b>##### INSERT YOUR CODE HERE #####</b>

#### I. AverageMeter
First we will write a helper function. AverageMeter is to calculate the mean of the running loss and accuracy.

*   It will have 2 functions which are reset and update.
*   reset will be called on initialization and set the attributes to 0.
*   update takes 2 arguments for the value and the size. It will add the value to the sum and the size to the count. Attribute "avg" (use this name) will also be updated as sum/count.

In [ ]:
class AverageMeter:
    """Computes and stores the average and current value"""

    def __init__(self):
      ##### START OF YOUR CODE #####
      self.reset()
    #Define the reset function.
    def reset(self):
      self.sum = 0
      self.count = 0
      self.avg = 0

    #Define the update function.
    def update(self,value,size):
      self.sum += value*size
      self.count += size
      self.avg = self.sum/self.count
      ##### END OF YOUR CODE #####

In [ ]:
avg_meter = AverageMeter()
avg_meter.update(100, 5)
avg_meter.update(50, 5)
print(avg_meter.avg, avg_meter.count)

75.0 10


#### II. Train Loop
Now we will write the training and validation loops. Detailed instructions are given within the code.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def training(train_loader, model, criterion, optimizer):
  # Let's start by initializing our AverageMeters.
  avg_meters = {'loss': AverageMeter(),
                'acc': AverageMeter()}

  # We will go through the train_loader.
  # Zero the gradients.
  # Make prediction.
  # Calculate the loss and the accuracy using prediction and labels.
  # Update the average meters.
  # Compute gradients and adjust learning weights.

  ##### START OF YOUR CODE #####
  #Set the train mode.
  model.train()
  model.to(device)
  #Seperate the inputs and labels.
  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    labels=labels.float()
  # Zero the gradients
  optimizer.zero_grad()
  #Make the forward pass.
  outputs = model(inputs)
  #Calculate the loss and accuracy
  loss = criterion(outputs,labels)
  predicted, _ = torch.max(outputs, 1)
  predicted = (predicted > 0.5).float()
  correct = (predicted.view(-1) == labels.view(-1)).sum().item()
  acc = correct / labels.size()[0]  #Update the average meters.
  #Update the average meters.
  avg_meters['loss'].update(loss.item(), labels.size(0))
  avg_meters['acc'].update(acc, labels.size(0))
  #Make the backward pass.
  loss.backward()
  #Compute gradients and adjust learning weights.
  optimizer.step()
  ##### END OF YOUR CODE #####

  return dict([('loss', avg_meters['loss'].avg),
                ('acc', avg_meters['acc'].avg)])

def validation(val_loader, model, criterion):
  avg_meters = {'loss': AverageMeter(),
                'acc': AverageMeter()}

  # Validation is almost the same but don't forget to turn the eval mode of the model and with torch no_grad.
  # You don't need to compute gradients or adjust learning weights for evaluation.

  ##### START OF YOUR CODE #####
  #Set the validation mode.
  model.eval()
  model.to(device)
  #Seperate the inputs and labels.
  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      labels=labels.float()
  #Make the forward pass.
  outputs = model(inputs)
  #Calculate the loss and accuracy
  loss = criterion(outputs,labels)
  predicted, _ = torch.max(outputs.data, 1)
  predicted = (predicted > 0.5).float()
  correct = (predicted.view(-1) == labels.view(-1)).sum().item()
  acc = correct / labels.size()[0]  #Update the average meters.
  avg_meters['loss'].update(loss.item(), labels.size(0))
  avg_meters['acc'].update(acc, labels.size(0))
  ##### END OF YOUR CODE #####

  return dict([('loss', avg_meters['loss'].avg),
              ('acc', avg_meters['acc'].avg)])

We will use Weights & Biases for hyperparameter search. This will only be an introduction and we highly recommend you to read the <a href="https://docs.wandb.ai/?_gl=1*1xon9b*_ga*NDg5OTYzNTM3LjE2NzUwNjYzNjk.*_ga_JH1SJHJQXJ*MTY3Njc0MDEyNi4xMi4xLjE2NzY3NDAxMjguNTguMC4w">documentation</a> for more information.

In [ ]:
!pip install wandb

In [ ]:
from torch import optim
import wandb

from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau

def main():
  # Set the initial configuration
  initial_config = {
      "data_dir": "/tmp/curated_data/data/",
      "image_size": 128,
      "train_batch_size": 64,
      "val_batch_size": 32,
      "test_batch_size": 1,
      "activation": "relu",
      "drop_rate": .2,
      "optimizer": "Adam",
      "learning_rate": 1e-3,
      "l2_reg": 1e-4, # Weight decay
      "nb_epoch": 50,
      "early_stopping": 15, # trigger value for early stopping
      }
  # Using this configuration dictionary:
  # initialize wandb
  # Create a run directory in your drive ("/content/drive/MyDrive/CMPE_runs/" + the current run name that you'll get from wandb)
  # Create the model
  # Create dataloader dictionary with "train", "val", "test" keys
  # Define binary cross entropy loss
  # Define optimizer with weight decay
  # Set lr scheduler to ReduceLROnPlateau:
    # It will decrease the lr by .1 if the val_loss did not decrease > .01. The minimum lr value can be 1e-9.
  # Print train and val results and log them to wandb at the end of each epoch
  # Save best model weights to your run directory when the val accuracy is at least .01 better than the best val accuracy.
  # Set early stopping with the trigger in config["early_stopping"], monitoring val accuracy. config["early_stopping"] = -1 means no early stopping.
  # Print when a new model is saved or early stopping trigger is reached.
  # After the final epoch (or early stopping), load the best model weights and log the test results to wandb

  ##### START OF YOUR CODE #####
  #Initialize wandb.
  wandb.init(config=initial_config, project="CMPE_49T_Assignment3_Fatmanur_Yaman")

  #Create a run directory.
  run_dir = f"/content/drive/MyDrive/CMPE_runs/{wandb.run.name}"
  os.makedirs(run_dir, exist_ok=True)

  #Create the model.
  model = ResNet18(initial_config["activation"],initial_config["drop_rate"],initial_config["image_size"])
  model = model.to('cuda:0')

  #Create the data loader.
  #First get the train, val, and test datasets by using CustomDataset.
  train_dataset = CustomDataset(image_size=initial_config["image_size"], data_folder=initial_config["data_dir"], partition="train")
  val_dataset = CustomDataset(image_size=initial_config["image_size"], data_folder=initial_config["data_dir"], partition="val")
  test_dataset = CustomDataset(image_size=initial_config["image_size"], data_folder=initial_config["data_dir"], partition="test")
  #Then, load the data according to the batch size and dataset given above by using DataLoader.
  train_loader = DataLoader(train_dataset, batch_size=initial_config["train_batch_size"], shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=initial_config["val_batch_size"], shuffle=False)
  test_loader = DataLoader(test_dataset, batch_size=initial_config["test_batch_size"], shuffle=False)


  #Define binary entropy loss.
  criterion = torch.nn.BCELoss().to('cuda')

  #Define optimizer with weight decay. Use the learning rate and weight decay defined above.
  optimizer = optim.Adam(model.parameters(), lr=initial_config["learning_rate"], weight_decay=initial_config["l2_reg"])
  lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5,
                                  factor=0.1, verbose=True, min_lr=1e-9)
  trigger = 0
  best_val_acc = 0
  #Train and validate the model for nb_epoch times defined above.
  for epoch in range(initial_config["nb_epoch"]):
    print("epoch:", epoch + 1)
    # Training
    model.train()
    train_results = training(train_loader, model, criterion, optimizer)

    # Validation
    val_results = validation(val_loader, model, criterion)

    # Print the results at the end of each iteration.
    wandb.log({"train_loss": train_results["loss"], "train_acc": train_results["acc"],
                "val_loss": val_results["loss"], "val_acc": val_results["acc"]})
    # Adjust learning rate
    lr_scheduler.step(val_results["loss"])

    num_epochs = str(initial_config["nb_epoch"])
    print(f"Epoch {epoch + 1}/{num_epochs} - "
          f"Train Loss: {train_results['loss']:.4f}, Train Acc: {train_results['acc']:.4f} - "
          f"Val Loss: {val_results['loss']:.4f}, Val Acc: {val_results['acc']:.4f}")

    # Check if the current model is the best so far
    if val_results["acc"] > best_val_acc + 0.01:
      best_val_acc = val_results["acc"]
      # Save the model weights
      torch.save(model.state_dict(), os.path.join(run_dir, "best_model.pth"))
      print("New best model saved." + f"Epoch {epoch + 1}" +f"With accuracy {best_val_acc}")

    if val_results["acc"] <= best_val_acc:
      trigger += 1
    else:
      trigger = 0

    # Check for early stopping trigger
    if trigger >= initial_config["early_stopping"]:
      print("Early stopping triggered.")
      break


  # Load the best model weights for testing
  model.load_state_dict(torch.load(os.path.join(run_dir, "best_model.pth")))

  # Testing
  model.eval()
  test_results = validation(test_loader, model, criterion)

  # Log test results to wandb
  wandb.log({"test_loss": test_results["loss"], "test_acc": test_results["acc"]})

  # Print final test results
  print("Final Test Results:", test_results)

  ##### END OF YOUR CODE #####

In [ ]:
# Define the parameters that we will fine tune with, which are:
  # Activation function
  # Optimizer
  # Drop rate: should be chosen randomly from a uniform distribution between [0., 0.9]
  # Weight decay: should be chosen randomly from a uniform distribution between [0., 0.1]
  # Learning rate: should be chosen randomly from a uniform distribution between [0.0001, 0.1]
import random
parameter_dict = {
    ##### START OF YOUR CODE #####
    "activation": {"values": ["relu", "sigmoid", "tanh"]},
    "optimizer": {"values": ["adam", "sgd"]},
    "drop_rate": {"min": 0.0, "max": 0.9},
    "weight_decay": {"min": 0.0,"max": 0.1},
    "learning_rate": {"min": 0.0001,"max": 0.1},
    ##### END OF YOUR CODE #####
}

# Define a sweep configuration tells wandb that it will randomly choose from parameter dict and the purpose is to maximize val_accuracy
sweep_config = {
    ##### START OF YOUR CODE #####
    "method": "random",  # Use random search
    "name": "hyperparameter_tuning",  # Name of the sweep
    "metric": {"goal": "maximize", "name": "val_acc"},  # Objective to maximize
    "parameters": parameter_dict,
    ##### END OF YOUR CODE #####
}
sweep_config['parameters'] = parameter_dict

# Start the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="CMPE_49T_Assignment3_Fatmanur_Yaman")


Create sweep with ID: fyk6q1vc
Sweep URL: https://wandb.ai/fatmanuryaman/CMPE_49T_Assignment3_Fatmanur_Yaman/sweeps/fyk6q1vc


In [ ]:
wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: kzmf9z59 with config:
wandb: 	activation: tanh
wandb: 	drop_rate: 0.578297055589875
wandb: 	learning_rate: 0.01718733059869951
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.029699224936632383
wandb: Currently logged in as: fatmanuryaman. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


epoch: 1
Epoch 1/50 - Train Loss: 0.7561, Train Acc: 0.4000 - Val Loss: 0.6883, Val Acc: 0.7143
New best model saved.Epoch 1With accuracy 0.7142857142857143
epoch: 2
Epoch 2/50 - Train Loss: 0.6828, Train Acc: 0.4400 - Val Loss: 0.7327, Val Acc: 0.2857
epoch: 3
Epoch 3/50 - Train Loss: 1.0911, Train Acc: 0.6800 - Val Loss: 0.7193, Val Acc: 0.2857
epoch: 4
Epoch 4/50 - Train Loss: 0.4802, Train Acc: 0.8400 - Val Loss: 0.7111, Val Acc: 0.2857
epoch: 5
Epoch 5/50 - Train Loss: 0.8955, Train Acc: 0.8400 - Val Loss: 0.7972, Val Acc: 0.2857
epoch: 6
Epoch 6/50 - Train Loss: 0.4051, Train Acc: 0.8000 - Val Loss: 1.0310, Val Acc: 0.2857
epoch: 7
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
Epoch 7/50 - Train Loss: 0.7571, Train Acc: 0.6000 - Val Loss: 1.1798, Val Acc: 0.2857
epoch: 8
Epoch 8/50 - Train Loss: 0.7919, Train Acc: 0.6000 - Val Loss: 1.2399, Val Acc: 0.2857
epoch: 9
Epoch 9/50 - Train Loss: 0.6623, Train Acc: 0.5600 - Val Loss: 1.2166, Val Acc: 0.2857
epoch: 10
Epo

test_acc,▁
test_loss,▁
train_acc,▁▂▅██▇▄▄▄▅▁▅▇▄▅
train_loss,▅▄█▂▆▁▅▅▄▅▅▄▂▄▄
val_acc,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▂▁▁▂▅▇██▇▇▇▇▇▇
test_acc,1.0
test_loss,0.68177
train_acc,0.68
train_loss,0.73154
val_acc,0.28571


wandb: Agent Starting Run: vaiarq62 with config:
wandb: 	activation: tanh
wandb: 	drop_rate: 0.6672424104393866
wandb: 	learning_rate: 0.09147840473768462
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.06282594883633848
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


epoch: 1
Epoch 1/50 - Train Loss: 0.7064, Train Acc: 0.3600 - Val Loss: 0.6916, Val Acc: 0.5714
New best model saved.Epoch 1With accuracy 0.5714285714285714
epoch: 2
Epoch 2/50 - Train Loss: 1.0877, Train Acc: 0.6400 - Val Loss: 0.7058, Val Acc: 0.4286
epoch: 3
Epoch 3/50 - Train Loss: 1.6607, Train Acc: 0.5200 - Val Loss: 0.6949, Val Acc: 0.4286
epoch: 4
Epoch 4/50 - Train Loss: 1.2781, Train Acc: 0.6000 - Val Loss: 0.7253, Val Acc: 0.4286
epoch: 5
Epoch 5/50 - Train Loss: 0.7559, Train Acc: 0.5600 - Val Loss: 0.7433, Val Acc: 0.4286
epoch: 6
Epoch 6/50 - Train Loss: 0.6193, Train Acc: 0.6800 - Val Loss: 0.7520, Val Acc: 0.4286
epoch: 7
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
Epoch 7/50 - Train Loss: 0.5968, Train Acc: 0.7200 - Val Loss: 0.7693, Val Acc: 0.4286
epoch: 8
Epoch 8/50 - Train Loss: 0.4389, Train Acc: 0.8800 - Val Loss: 0.7233, Val Acc: 0.4286
epoch: 9
Epoch 9/50 - Train Loss: 0.6081, Train Acc: 0.7600 - Val Loss: 0.7069, Val Acc: 0.5714
epoch: 10
Epo

...

epoch: 1
Epoch 1/50 - Train Loss: 0.6708, Train Acc: 0.5200 - Val Loss: 0.6985, Val Acc: 0.4286
New best model saved.Epoch 1With accuracy 0.42857142857142855
epoch: 2
Epoch 2/50 - Train Loss: 1.6766, Train Acc: 0.6000 - Val Loss: 0.6885, Val Acc: 0.5714
New best model saved.Epoch 2With accuracy 0.5714285714285714
epoch: 3
Epoch 3/50 - Train Loss: 0.6760, Train Acc: 0.6800 - Val Loss: 0.7183, Val Acc: 0.4286
epoch: 4
Epoch 4/50 - Train Loss: 1.3771, Train Acc: 0.6400 - Val Loss: 0.7135, Val Acc: 0.4286
epoch: 5
Epoch 5/50 - Train Loss: 1.2401, Train Acc: 0.3200 - Val Loss: 0.7022, Val Acc: 0.4286
epoch: 6
Epoch 6/50 - Train Loss: 0.5925, Train Acc: 0.7200 - Val Loss: 0.7135, Val Acc: 0.4286
epoch: 7
Epoch 7/50 - Train Loss: 1.0555, Train Acc: 0.4400 - Val Loss: 0.7525, Val Acc: 0.4286
epoch: 8
Epoch 00008: reducing learning rate of group 0 to 1.0000e-04.
Epoch 8/50 - Train Loss: 0.6523, Train Acc: 0.7200 - Val Loss: 0.7438, Val Acc: 0.4286
epoch: 9
Epoch 9/50 - Train Loss: 0.7540, Train

test_acc,▁
test_loss,▁
train_acc,▄▅▆▆▁▇▃▇▅▅█▆▅▄▇
train_loss,▂█▂▆▅▁▄▁▂▂▁▁▂▂▁
val_acc,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▁▄▄▂▄█▇▆▅▄▄▃▃▃
test_acc,0.0
test_loss,0.73145
train_acc,0.76
train_loss,0.63252
val_acc,0.42857


wandb: Agent Starting Run: wh1vuw3l with config:
wandb: 	activation: tanh
wandb: 	drop_rate: 0.266736652895666
wandb: 	learning_rate: 0.09168045569689738
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.07026993616919426
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


epoch: 1
Epoch 1/50 - Train Loss: 0.7128, Train Acc: 0.4400 - Val Loss: 0.6947, Val Acc: 0.4286
New best model saved.Epoch 1With accuracy 0.42857142857142855
epoch: 2


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
!pip install nbconvert
from nbconvert import HTMLExporter
import nbformat

notebook_file = "Fatmanur_Yaman_CMPE_49T_Assignment3.ipynb"

with open(notebook_file) as f:
    nb = nbformat.read(f, as_version=4)

html_exporter = HTMLExporter()
html_exporter.template_name = 'classic'
body, resources = html_exporter.from_notebook_node(nb)

with open('Fatmanur_Yaman_CMPE_49T_Assignment3.html', 'w', encoding='utf-8') as f:
    f.write(body)